In [2]:
import emoji
import numpy as np

In [3]:
#define emoji dictionary used
emoji_dictionary = {"0" : "\u2764\uFE0F",
                    "1" : ":baseball:",
                    "2" : ":grinning_face_with_big_eyes:",
                    "3" : ":disappointed_face:",
                    "4" : ":fork_and_knife:"
                   }


In [4]:
for e in emoji_dictionary:
    print(emoji.emojize(emoji_dictionary[e]))

❤️
⚾
😃
😞
🍴


In [5]:
def process_x(path):
    f = open(path)
    text = f.read()
    f.close()
    text = text.replace("' " , "',")
    text = text.replace("\n", ',')
    sep = text.split(",")
    for i,sent in enumerate(sep):
        sep[i] = sent.replace("'", "")
        sep[i] = sep[i].split()
    x = np.array(sep)
    return x

def process_y(path):
    f = open(path)
    y = f.read()
    f.close()
    y = y.split(" ")
    y_ = []
    for i, yval in enumerate(y):
        y[i] = yval.replace("\n", "")
        y_.append(y[i])
    return np.array(y_)

In [6]:
X_TRAIN_PATH = 'emojify_train_x.csv'
Y_TRAIN_PATH = 'Emojify_Y_train.csv'
X_TEST_PATH =  'emojiy_test_x.csv'
Y_TEST_PATH =  'emojiy_y_test.csv'

In [7]:
x_train = process_x(X_TRAIN_PATH)
y_train = process_y(Y_TRAIN_PATH)

x_test = process_x(X_TEST_PATH)
y_test = process_y(Y_TEST_PATH)

c:\users\rahul\anaconda3\envs\gpu1\lib\site-packages\ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # This is added back by InteractiveShellApp.init_path()


In [8]:
x_train[:5]

array([list(['never', 'talk', 'to', 'me', 'again']),
       list(['I', 'am', 'proud', 'of', 'your', 'achievements']),
       list(['It', 'is', 'the', 'worst', 'day', 'in', 'my', 'life']),
       list(['Miss', 'you', 'so', 'much']), list(['food', 'is', 'life'])],
      dtype=object)

In [9]:
for i in range(5):
    print(x_train[i], emoji.emojize(emoji_dictionary[y_train[i]]))

['never', 'talk', 'to', 'me', 'again'] 😞
['I', 'am', 'proud', 'of', 'your', 'achievements'] 😃
['It', 'is', 'the', 'worst', 'day', 'in', 'my', 'life'] 😞
['Miss', 'you', 'so', 'much'] ❤️
['food', 'is', 'life'] 🍴


In [10]:
# get word embeddings 

f = open('glove.6B.50d.txt', encoding = 'utf-8')
embeddings = {}

for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float')
    embeddings[word] = coefs

In [11]:
#convert sentence to word vectors and create 3d tensor input for rnn
#length of word vectors to form tensor
emb_dim = list(embeddings.values())[0].shape[0]

def embedding_output(x):
    maxLen = 10
    embedding_out = np.zeros((x.shape[0], maxLen, emb_dim))
    
    #iterate over sentences
    for ix in range(x.shape[0]):
        
        #iterate over list of words for every sentence
        for ij in range(len(x[ix])):
            try:
                embedding_out [ix][ij] = embeddings[x[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    return embedding_out

In [12]:
#form the matrix from input data

x_train_embedded_matrix = embedding_output(x_train)
x_test_embedded_matrix = embedding_output(x_test)

In [13]:
print(x_train_embedded_matrix.shape, x_test_embedded_matrix.shape)

(132, 10, 50) (56, 10, 50)


In [14]:
#convert y to one hot
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


Using TensorFlow backend.


In [15]:
print(y_train.shape, y_test.shape)

(132, 5) (56, 5)


In [16]:
#create model
from keras.models import Sequential
from keras.layers import *

In [24]:
model = Sequential()
model.add(LSTM(64, input_shape = (10, 50)))
model.add(Dropout(0.5))

model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [25]:
modelstack = Sequential()
modelstack.add(LSTM(64, input_shape = (10, 50), return_sequences = True))
modelstack.add(Dropout(0.5))
modelstack.add(LSTM(64, return_sequences = False))
modelstack.add(Dropout(0.5))
modelstack.add(Dense(5))
modelstack.add(Activation('softmax'))

modelstack.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelstack.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [27]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5", monitor = 'val_loss', verbose = True, save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_acc', patience = 5)

model.fit(x_train_embedded_matrix, y_train, epochs = 100, batch_size= 64, shuffle = True, validation_split = 0.2, callbacks = [checkpoint, earlystop])

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 0s 3ms/step - loss: 0.2903 - accuracy: 0.8952 - val_loss: 1.1776 - val_accuracy: 0.6296

Epoch 00001: val_loss improved from inf to 1.17761, saving model to best_model.h5
Epoch 2/100
105/105 [==============================] - 0s 218us/step - loss: 0.1840 - accuracy: 0.9524 - val_loss: 1.0371 - val_accuracy: 0.5556

Epoch 00002: val_loss improved from 1.17761 to 1.03707, saving model to best_model.h5
Epoch 3/100
105/105 [==============================] - 0s 200us/step - loss: 0.2068 - accuracy: 0.9429 - val_loss: 1.0828 - val_accuracy: 0.5926

Epoch 00003: val_loss did not improve from 1.03707
Epoch 4/100
105/105 [==============================] - 0s 217us/step - loss: 0.2232 - accuracy: 0.9429 - val_loss: 1.1855 - val_accuracy: 0.5926

Epoch 00004: val_loss did not improve from 1.03707
Epoch 5/100
105/105 [==============================] - 0s 228us/step - loss: 0.1773 - accuracy: 0.9524 

c:\users\rahul\anaconda3\envs\gpu1\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning



Epoch 00007: val_loss did not improve from 1.03707
Epoch 8/100
105/105 [==============================] - 0s 228us/step - loss: 0.1328 - accuracy: 0.9619 - val_loss: 1.0376 - val_accuracy: 0.5185

Epoch 00008: val_loss did not improve from 1.03707
Epoch 9/100
105/105 [==============================] - 0s 230us/step - loss: 0.1324 - accuracy: 0.9619 - val_loss: 1.0932 - val_accuracy: 0.5185

Epoch 00009: val_loss did not improve from 1.03707
Epoch 10/100
105/105 [==============================] - 0s 249us/step - loss: 0.1012 - accuracy: 0.9810 - val_loss: 1.1855 - val_accuracy: 0.5556

Epoch 00010: val_loss did not improve from 1.03707
Epoch 11/100
105/105 [==============================] - 0s 237us/step - loss: 0.1202 - accuracy: 0.9714 - val_loss: 1.2318 - val_accuracy: 0.5185

Epoch 00011: val_loss did not improve from 1.03707
Epoch 12/100
105/105 [==============================] - 0s 257us/step - loss: 0.0883 - accuracy: 0.9810 - val_loss: 1.3041 - val_accuracy: 0.5556

Epoch 00012

105/105 [==============================] - 0s 285us/step - loss: 0.0094 - accuracy: 1.0000 - val_loss: 1.4199 - val_accuracy: 0.5556

Epoch 00049: val_loss did not improve from 1.03707
Epoch 50/100
105/105 [==============================] - 0s 313us/step - loss: 0.0176 - accuracy: 1.0000 - val_loss: 1.4535 - val_accuracy: 0.5556

Epoch 00050: val_loss did not improve from 1.03707
Epoch 51/100
105/105 [==============================] - 0s 294us/step - loss: 0.0167 - accuracy: 1.0000 - val_loss: 1.5278 - val_accuracy: 0.5556

Epoch 00051: val_loss did not improve from 1.03707
Epoch 52/100
105/105 [==============================] - 0s 323us/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 1.5468 - val_accuracy: 0.5556

Epoch 00052: val_loss did not improve from 1.03707
Epoch 53/100
105/105 [==============================] - 0s 285us/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 1.5147 - val_accuracy: 0.5185

Epoch 00053: val_loss did not improve from 1.03707
Epoch 54/100
105/105 [=

105/105 [==============================] - 0s 285us/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 1.7860 - val_accuracy: 0.5556

Epoch 00090: val_loss did not improve from 1.03707
Epoch 91/100
105/105 [==============================] - 0s 285us/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 1.8238 - val_accuracy: 0.5556

Epoch 00091: val_loss did not improve from 1.03707
Epoch 92/100
105/105 [==============================] - 0s 289us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 1.8707 - val_accuracy: 0.5556

Epoch 00092: val_loss did not improve from 1.03707
Epoch 93/100
105/105 [==============================] - 0s 297us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 1.9236 - val_accuracy: 0.5556

Epoch 00093: val_loss did not improve from 1.03707
Epoch 94/100
105/105 [==============================] - 0s 313us/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 1.9646 - val_accuracy: 0.5556

Epoch 00094: val_loss did not improve from 1.03707
Epoch 95/100
105/105 [=

In [28]:


checkpoint = ModelCheckpoint("best_model_stack.h5", monitor = 'val_loss', verbose = True, save_best_only = True)
earlystop = EarlyStopping(monitor = 'val_acc', patience = 5)

modelstack.fit(x_train_embedded_matrix, y_train, epochs = 100, batch_size= 64, shuffle = True, validation_split = 0.2, callbacks = [checkpoint, earlystop])

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 1s 7ms/step - loss: 0.3353 - accuracy: 0.8952 - val_loss: 1.0385 - val_accuracy: 0.6296

Epoch 00001: val_loss improved from inf to 1.03850, saving model to best_model_stack.h5
Epoch 2/100
105/105 [==============================] - 0s 335us/step - loss: 0.2855 - accuracy: 0.9238 - val_loss: 1.0612 - val_accuracy: 0.6667

Epoch 00002: val_loss did not improve from 1.03850
Epoch 3/100
105/105 [==============================] - 0s 361us/step - loss: 0.2304 - accuracy: 0.9429 - val_loss: 0.9597 - val_accuracy: 0.7037

Epoch 00003: val_loss improved from 1.03850 to 0.95968, saving model to best_model_stack.h5
Epoch 4/100
105/105 [==============================] - 0s 351us/step - loss: 0.2639 - accuracy: 0.9238 - val_loss: 0.9816 - val_accuracy: 0.7037

Epoch 00004: val_loss did not improve from 0.95968
Epoch 5/100
105/105 [==============================] - 0s 358us/step - loss: 0.2294 - accur

c:\users\rahul\anaconda3\envs\gpu1\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning



Epoch 00005: val_loss did not improve from 0.95968
Epoch 6/100
105/105 [==============================] - 0s 359us/step - loss: 0.2211 - accuracy: 0.9524 - val_loss: 1.0501 - val_accuracy: 0.7407

Epoch 00006: val_loss did not improve from 0.95968
Epoch 7/100
105/105 [==============================] - 0s 371us/step - loss: 0.2030 - accuracy: 0.9429 - val_loss: 1.1912 - val_accuracy: 0.7037

Epoch 00007: val_loss did not improve from 0.95968
Epoch 8/100
105/105 [==============================] - 0s 381us/step - loss: 0.2006 - accuracy: 0.9524 - val_loss: 1.3491 - val_accuracy: 0.6296

Epoch 00008: val_loss did not improve from 0.95968
Epoch 9/100
105/105 [==============================] - 0s 465us/step - loss: 0.1683 - accuracy: 0.9619 - val_loss: 1.1695 - val_accuracy: 0.7037

Epoch 00009: val_loss did not improve from 0.95968
Epoch 10/100
105/105 [==============================] - 0s 465us/step - loss: 0.2085 - accuracy: 0.9619 - val_loss: 1.1097 - val_accuracy: 0.7407

Epoch 00010: 

105/105 [==============================] - 0s 399us/step - loss: 0.0186 - accuracy: 1.0000 - val_loss: 1.2179 - val_accuracy: 0.7407

Epoch 00047: val_loss did not improve from 0.95968
Epoch 48/100
105/105 [==============================] - 0s 389us/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 1.3032 - val_accuracy: 0.7037

Epoch 00048: val_loss did not improve from 0.95968
Epoch 49/100
105/105 [==============================] - 0s 361us/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 1.2869 - val_accuracy: 0.7037

Epoch 00049: val_loss did not improve from 0.95968
Epoch 50/100
105/105 [==============================] - 0s 361us/step - loss: 0.0210 - accuracy: 1.0000 - val_loss: 1.2501 - val_accuracy: 0.7037

Epoch 00050: val_loss did not improve from 0.95968
Epoch 51/100
105/105 [==============================] - 0s 370us/step - loss: 0.0153 - accuracy: 1.0000 - val_loss: 1.2981 - val_accuracy: 0.7037

Epoch 00051: val_loss did not improve from 0.95968
Epoch 52/100
105/105 [=

105/105 [==============================] - 0s 370us/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 1.6420 - val_accuracy: 0.7037

Epoch 00088: val_loss did not improve from 0.95968
Epoch 89/100
105/105 [==============================] - 0s 370us/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 1.6375 - val_accuracy: 0.7037

Epoch 00089: val_loss did not improve from 0.95968
Epoch 90/100
105/105 [==============================] - 0s 441us/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 1.6258 - val_accuracy: 0.7037

Epoch 00090: val_loss did not improve from 0.95968
Epoch 91/100
105/105 [==============================] - 0s 463us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 1.6009 - val_accuracy: 0.7037

Epoch 00091: val_loss did not improve from 0.95968
Epoch 92/100
105/105 [==============================] - 0s 408us/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 1.5864 - val_accuracy: 0.7037

Epoch 00092: val_loss did not improve from 0.95968
Epoch 93/100
105/105 [=

In [30]:
model.load_weights("best_model.h5")
modelstack.load_weights("best_model_stack.h5")

In [31]:
pred = model.predict_classes(x_test_embedded_matrix)
predstack = modelstack.predict_classes(x_test_embedded_matrix)

In [32]:

acc = model.evaluate(x_test_embedded_matrix, y_test)
print('normal model gives -  ', round(acc[1],2) *100)

accstack = modelstack.evaluate(x_test_embedded_matrix, y_test)
print('stacked model gives -  ', round(accstack[1], 2)*100)

56/56 [==============================] - 0s 160us/step
normal model gives -   68.0
56/56 [==============================] - 0s 213us/step
stacked model gives -   56.99999999999999


In [27]:
pred

array([3, 3, 2, 0, 2, 2, 3, 2, 1, 2, 1, 2, 0, 3, 1, 3, 2, 2, 3, 3, 0, 0,
       4, 2, 3, 3, 1, 0, 1, 2, 0, 1, 3, 2, 3, 1, 2, 4, 1, 0, 1, 0, 2, 0,
       2, 0, 3, 2, 3, 1, 3, 0, 3, 2, 0, 0], dtype=int64)

In [61]:
for i in pred:
    print(emoji.emojize(emoji_dictionary[str(i)]))

😞
😞
😃
❤️
😃
😃
😞
😃
⚾
😃
⚾
😃
❤️
😞
⚾
😞
😃
😃
😞
😞
❤️
😞
🍴
😃
😞
😞
❤️
❤️
⚾
😃
❤️
⚾
😞
😃
😞
⚾
😃
🍴
⚾
❤️
⚾
❤️
❤️
❤️
😃
❤️
😞
😃
😞
⚾
😞
❤️
😞
😃
❤️
❤️
ERROR! Session/line number was not unique in database. History logging moved to new session 89


In [66]:
with open('emojiy_y_test.csv') as f:
    y_t = f.read().split()
for i in pred:
    print(emoji.emojize(emoji_dictionary[str(i)]))

😞
😞
😃
❤️
😃
😃
😞
😃
⚾
😃
⚾
😃
❤️
😞
⚾
😞
😃
😃
😞
😞
❤️
😞
🍴
😃
😞
😞
❤️
❤️
⚾
😃
❤️
⚾
😞
😃
😞
⚾
😃
🍴
⚾
❤️
⚾
❤️
❤️
❤️
😃
❤️
😞
😃
😞
⚾
😞
❤️
😞
😃
❤️
❤️


In [64]:
sent = [[' '.join(i)] for i in x_test]
for idx in range(len(sent)):
    sent[idx][0] = sent[idx][0].replace('\\t', '')

In [69]:
print("Sentence")
print("actual")
print("predicted")
for idx,[line] in enumerate(sent):
    print(line)
    print(emoji.emojize(emoji_dictionary[y_t[idx]]))
    print(emoji.emojize(emoji_dictionary[str(pred[idx])]))

Sentence
actual
predicted
I want to eat
🍴
😞
he did not answer
😞
😞
he got a very nice raise
😃
😃
she got me a nice present
😃
❤️
ha ha ha it was so funny
😃
😃
he is a good friend
😃
😃
I am upset
😞
😞
We had such a lovely dinner tonight
😃
😃
where is the food
🍴
⚾
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious
😞
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working 
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop pissing me off
😞
😞
any suggestions for dinner
🍴
😞
I love taking breaks
❤️
❤️
you brighten my day
😃
😞
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
give me the ball
⚾
❤️
My grandmother is the love of my life
❤️
❤️
enjoy your game
⚾
⚾
valentine day is near
😃
😃
I miss you so much
❤️
❤️
throw the ball
⚾
⚾
My life is so boring
😞
😞
she said yes
😃
😃
will you be my valentine
😃
😞
he can pitch really well
⚾
⚾
dance with me
😃
😃
I am hungry
🍴
🍴
See you at the 